In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 13.1 MB/s eta 0:00:00


In [3]:
# Import libraries
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0, ResNet101
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Save | Load trained model

In [14]:
#model.save_weights('/content/drive/MyDrive/SHSU-Projects/Final_files/Dataset_2_after_augmented_convxNxtBase_model_weights.weight')
model.load_weights('/content/drive/MyDrive/SHSU-Projects/Final_files/Dataset_2_after_augmented_convxNxtBase_model_weights.weight')


## Save tflite model

In [15]:
model.save('model_saved_dir')     # saving model temp in  directory format
converter = tf.lite.TFLiteConverter.from_saved_model('model_saved_dir')

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


tflite_model_path = '/content/drive/MyDrive/SHSU-Projects/Final_files/Dataset_2_after_augmented_convxNxtBase_model_tflite.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)


print('TFLite model saved at:', tflite_model_path)

TFLite model saved at: /content/drive/MyDrive/SHSU-Projects/Final_files/Dataset_2_after_augmented_convxNxtBase_model_tflite.tflite


## Do the inference using tflite (Testing)

In [23]:
import tensorflow as tf
import cv2

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Assuming your input image is in 'input_image.jpg'
#input_image = '/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_2_Original/test/0Normal/NormalG0 (433).png'
# input_image = '/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_2_Original/test/1Doubtful/DoubtfulG1 (433).png'
input_image = '/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_2_Original/test/2Mild/MildG2 (172).png'
# input_image = '/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_2_Original/test/3Moderate/ModerateG3 (162).png'
# input_image = '/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_2_Original/test/4Severe/SevereG4 (192).png'

rgb_image = cv2.cvtColor(cv2.imread(input_image), cv2.COLOR_BGR2RGB)
rgb_image = tfa.image.equalize(rgb_image)

input_data = tf.image.resize(rgb_image, IMAGE_SIZE)
input_data = tf.cast(input_data, tf.float32)
input_data = tf.expand_dims(input_data, axis=0)

input_data = tf.keras.applications.resnet50.preprocess_input(input_data)

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])

predicted_class = tf.argmax(output_data[0]).numpy()

print(f"Predicted class: {predicted_class}")


Predicted class: 2


## GUI (Standalone code)

In [17]:
!pip3 install gradio==3.43.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.6

In [19]:
pip install typing_extensions==4.7.1 --upgrade

  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
fastapi 0.104.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.7.1 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.7.1 which is incompatible.


In [20]:
import gradio as gr
import tensorflow as tf
import cv2

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/SHSU-Projects/Final_files/Dataset_2_after_augmented_convxNxtBase_model_tflite.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def classify_image(image):

    #image = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB)
    image = tfa.image.equalize(image)
    input_data = image.resize((256, 256))
    input_data = tf.cast(input_data, tf.float32)
    input_data = tf.expand_dims(input_data, axis=0)


    input_data = tf.keras.applications.resnet50.preprocess_input(input_data)

    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    predicted_class = tf.argmax(output_data[0]).numpy()

    print(f"Predicted class: {predicted_class}")

    class_list = ['Healthy', 'Doubtful', 'Minimal', 'Moderate', 'Severe']
    return class_list[predicted_class]



gr.Interface(fn=classify_image,
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=5)).launch(share=True)


ImportError: ignored